In [1]:
import pandas as pd
import numpy as np
import psycopg2


Connect to Database

In [2]:
from configparser import ConfigParser



def config(filename=f'database.ini', section='postgres'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [3]:
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
    # execute a statement
        print('PostgreSQL database version:')
        cur.execute('SELECT version()')

        # display the PostgreSQL database server version
        db_version = cur.fetchone()
        print(db_version)

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            return conn

get tables names

In [4]:
conn = connect()
cur = conn.cursor()

t_query = """SELECT TABLE_NAME
                FROM INFORMATION_SCHEMA.TABLES
                WHERE TABLE_SCHEMA = 'public'"""
cur.execute(t_query)
names = [names[0] for names in cur.fetchall()]
print(names)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 13.3 on x86_64-pc-linux-musl, compiled by gcc (Alpine 10.2.1_pre1) 10.2.1 20201203, 64-bit',)
['dic_units', 'dic_resource_names', 'dic_upper_works', 'dic_work_titles', 'json_projects', 'progress_ranges', 'res_progress_ranges', 'json_files', 'json_validation_files', 'json_validation_file_names', 'json_resources', 'json_validation_resources', 'json_works', 'json_validation_works']


Get Data To Numpy

In [5]:
def cut_tails(array, min, max):
    array_cut = []
    qmax, qmin = np.percentile(array, [max, min])
    
    interval = qmax - qmin

    min_dist = qmin - (1.5 * interval)
    max_dist = qmax + (1.5 * interval)
    
    if min_dist is None or max_dist is None:
        return array
    for data in array:
        if data < max_dist and data > min_dist:
            array_cut.append(data)
            
    return array_cut

In [6]:
query = "SELECT message from json_validation_resources"
source_df = pd.read_sql_query(query, con=conn)
print(source_df)


C:\Users\Anatoly\AppData\Local\Temp\ipykernel_15812\2260430895.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  source_df = pd.read_sql_query(query, con=conn)


        message
0          None
1          None
2          None
3          None
4          None
...         ...
1228585    None
1228586    None
1228587    None
1228588    None
1228589    None

[1228590 rows x 1 columns]
